In [33]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.layers import Embedding, GRU, Dense
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import tensorflow as tf
import pandas as pd
import numpy as np
import string

In [34]:
df = pd.read_csv("turkish_movie_sentiment_dataset.csv")
df.head()

,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\r\nbu adam...,Sevginin Gücü,"5,0"


In [35]:
df.shape

(83227, 3)

In [36]:
comments = lambda x : x[23:-24]

df["comment"] = df["comment"].apply(comments)
df["comment"].head()

0    Jean Reno denince zaten leon filmi gelir akla ...
1    Ekşın falan izlemek istiyorsanız eğer bunu izl...
2    Bu yapım hakkında öyle çok şey yazabilirim ki ...
3    finali yeter... (sting - shape of my heart)\r\...
4    Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...
Name: comment, dtype: object

In [37]:
floatize = lambda x : float(x[0:-2])

df["point"] = df["point"].apply(floatize)
df["point"].value_counts()

4.0    27463
5.0    15873
3.0    14494
2.0    13866
1.0     6381
0.0     5150
Name: point, dtype: int64

In [38]:
df["point"] = df["point"].replace(1, 0)
df["point"] = df["point"].replace(2, 0)
df["point"] = df["point"].replace(3, 1)
df["point"] = df["point"].replace(4, 1)
df["point"] = df["point"].replace(5, 1)
df["point"].value_counts()

1.0    57830
0.0    25397
Name: point, dtype: int64

In [39]:
df.reset_index(inplace = True)
df.drop("index", axis = 1, inplace = True)
df.head()

,comment,film_name,point
0,Jean Reno denince zaten leon filmi gelir akla ...,Sevginin Gücü,1.0
1,Ekşın falan izlemek istiyorsanız eğer bunu izl...,Sevginin Gücü,1.0
2,Bu yapım hakkında öyle çok şey yazabilirim ki ...,Sevginin Gücü,1.0
3,finali yeter... (sting - shape of my heart)\r\...,Sevginin Gücü,1.0
4,Jean Reno..\r\nbu adam kusursuz biri..\r\nve o...,Sevginin Gücü,1.0


In [40]:
df_0 = df[(df["point"] == 0.0)]
df_1 = df[(df["point"] == 1.0)]
df_1 = df.iloc[0:25397, :]

In [41]:
df = pd.concat([df_0, df_1])
df.shape

(50794, 3)

In [42]:
df["comment"] = df["comment"].apply(lambda x: x.lower())
df.head()

,comment,film_name,point
8,film güzel ama çok abartıldığını düşünüyorum\n...,Sevginin Gücü,0.0
20,diğer remake'lerden kalite olarak ne üstün ne ...,Teksas Katliamı 3D,0.0
21,"güzel bir devam filmi fikriyle yola çıkılmış, ...",Teksas Katliamı 3D,0.0
24,teksas katliamı serisi fanı olarak bu filmi va...,Teksas Katliamı 3D,0.0
40,bakalım 3 gün içinde tüm seriyi izlicem. filmi...,Baba 3,0.0


In [43]:
def remove_punctuation(text):
  no_punc = [words for words in text if words not in string.punctuation]
  word_wo_punc = "".join(no_punc)
  return word_wo_punc

df["comment"] = df["comment"].apply(lambda x: remove_punctuation(x))
df["comment"] = df["comment"].apply(lambda x: x.replace("\r", " "))
df["comment"] = df["comment"].apply(lambda x: x.replace("\n", " "))

df.head()

,comment,film_name,point
8,film güzel ama çok abartıldığını düşünüyorum ö...,Sevginin Gücü,0.0
20,diğer remakelerden kalite olarak ne üstün ne d...,Teksas Katliamı 3D,0.0
21,güzel bir devam filmi fikriyle yola çıkılmış a...,Teksas Katliamı 3D,0.0
24,teksas katliamı serisi fanı olarak bu filmi va...,Teksas Katliamı 3D,0.0
40,bakalım 3 gün içinde tüm seriyi izlicem filmi ...,Baba 3,0.0


In [44]:
def remove_numeric(corpus):
  output = "".join(words for words in corpus if not words.isdigit())
  return output

df["comment"] = df["comment"].apply(lambda x: remove_numeric(x)) 
df.head()

,comment,film_name,point
8,film güzel ama çok abartıldığını düşünüyorum ö...,Sevginin Gücü,0.0
20,diğer remakelerden kalite olarak ne üstün ne d...,Teksas Katliamı 3D,0.0
21,güzel bir devam filmi fikriyle yola çıkılmış a...,Teksas Katliamı 3D,0.0
24,teksas katliamı serisi fanı olarak bu filmi va...,Teksas Katliamı 3D,0.0
40,bakalım gün içinde tüm seriyi izlicem filmi g...,Baba 3,0.0


In [45]:
#target = df["point"].values.tolist()
data = df["comment"].values.tolist()

#cutoff = int(len(data)*0.80)

#x_train, x_test = data[:cutoff], data[cutoff:]
#y_train, y_test = target[:cutoff], target[cutoff:]

#x_train, x_val = x_train[:cutoff], x_train[cutoff:]
#y_train, y_val = y_train[:cutoff], y_train[cutoff:]

train, validate, test = np.split(df.sample(frac = 1, random_state = 42), [int(.6*len(df)), int(.8*len(df))])

print(len(train)/len(data), len(test)/len(data), len(validate)/len(data))

0.5999921250541402 0.20000393747292988 0.20000393747292988


In [46]:
num_words = 15000
tokenizer = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts(data)

In [47]:
x_train_tokens = tokenizer.texts_to_sequences(train.comment)
x_test_tokens = tokenizer.texts_to_sequences(test.comment)
x_val_tokens = tokenizer.texts_to_sequences(validate.comment)

#print([x_train[200]])
print(x_train_tokens[200])

[1817, 99, 179, 1603, 996, 11, 249, 1, 541, 2266, 1162, 16, 226]


In [48]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
num_tokens

array([85, 13, 10, ..., 38, 13, 61])

In [49]:
np.mean(num_tokens)

32.54620401132029

In [50]:
np.max(num_tokens)

3402

In [51]:
max_tokens = np.mean(num_tokens) + (5*np.std(num_tokens))
max_tokens = int(max_tokens)
max_tokens

306

In [52]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.992469545957918

In [53]:
x_train_pad = pad_sequences(x_train_tokens, maxlen = max_tokens) 
x_test_pad = pad_sequences(x_test_tokens, maxlen = max_tokens)
x_val_pad = pad_sequences(x_val_tokens, maxlen = max_tokens)

In [54]:
np.array(x_train_tokens[1])

array([  185,   101,   136,    17,     1,     8, 12193,   181,     2,
          83,  1628,   405, 10215])

In [55]:
x_train_pad[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [56]:
#model.add(layers.LSTM(128))
#model.add(SimpleRNN(128))
#model.add(Dense(1, activation = "relu"))
#model.add(Dense(10, activation = "softmax"))

#model = Sequential([
#                    tf.keras.layers.Embedding(
#                        input_dim = num_words,
#                        output_dim = 25,
#                        mask_zero = True),
#                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(5, 10))),
#                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#                    tf.keras.layers.Dense(64),
#                    tf.keras.layers.Dense(1, activation = "tanh")
#])

embedding_size = 50
model = Sequential([
                   Embedding(input_dim = num_words, output_dim = embedding_size, input_length = max_tokens, name = "embedding_layer", mask_zero = True),
                   GRU(units = 16, 
                       activation = "tanh",
                       recurrent_activation = "sigmoid",
                       use_bias = True,
                       kernel_initializer = "glorot_uniform",
                       recurrent_initializer = "orthogonal",
                       bias_initializer = "zeros",
                       dropout = 0.1,
                       recurrent_dropout = 0.1,
                       return_sequences = True,
                       return_state = False),
                   GRU(units = 8, return_sequences = True),
                   GRU(units = 4),
                   Dense(4),
                   Dense(1, activation = "sigmoid")
                   ])

In [57]:
optimizer = Adam(lr = 1e-3)
model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 306, 50)           750000    
_________________________________________________________________
gru_3 (GRU)                  (None, 306, 16)           3216      
_________________________________________________________________
gru_4 (GRU)                  (None, 306, 8)            600       
_________________________________________________________________
gru_5 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 753,997
Trainable params: 753,997
Non-trainable params: 0
________________________________________________

In [58]:
x_train_pad = np.array(x_train_pad)
y_train = np.array(train.point)

x_val_pad = np.array(x_val_pad)
y_val = np.array(validate.point)

#model.fit(x_train_pad, y_train, validation_split=0.1, epochs = 5, batch_size = 256)
model.fit(x_train_pad, y_train, validation_data=(x_val_pad, y_val), epochs = 5, batch_size = 64)

Epoch 1/5
477/477 [==============================] - 379s 778ms/step - loss: 0.5920 - accuracy: 0.6776 - val_loss: 0.4889 - val_accuracy: 0.7717
Epoch 2/5
477/477 [==============================] - 370s 775ms/step - loss: 0.4060 - accuracy: 0.8226 - val_loss: 0.4958 - val_accuracy: 0.7689
Epoch 3/5
477/477 [==============================] - 373s 783ms/step - loss: 0.3355 - accuracy: 0.8633 - val_loss: 0.5247 - val_accuracy: 0.7625
Epoch 4/5
477/477 [==============================] - 379s 795ms/step - loss: 0.2717 - accuracy: 0.8957 - val_loss: 0.5666 - val_accuracy: 0.7584
Epoch 5/5
477/477 [==============================] - 381s 798ms/step - loss: 0.2250 - accuracy: 0.9203 - val_loss: 0.6545 - val_accuracy: 0.7554


In [59]:
model.save("RNN")

INFO:tensorflow:Assets written to: RNN/assets


In [60]:
modelRNN = tf.keras.models.load_model(
    "RNN", custom_objects=None, compile=True, options=None
)

In [61]:
y_test = np.array(test.point)
scores = modelRNN.evaluate(x_test_pad, y_test, verbose=0)

In [65]:
print("%s: %.2f%%" % (modelRNN.metrics_names[1], scores[1]*100))

accuracy: 76.54%


In [63]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = " ".join(words)
    return text

In [64]:
prediction = modelRNN.predict(x_test_pad)
print(tokens_to_string(x_test_pad[12]), prediction[12], y_test[12])
print(tokens_to_string(x_test_pad[500]), prediction[500], y_test[500])

muhtemel aşk adıyla ülkemizde yayınlanan bu film aslında alıştığımız bir aşk filmi değil tamamen sınıfının hayatına odaklı geçen ve neredeyse aşk hariç her insanın gözüne bir film olmuş ancak filmin genel akışı kötüydü ve beni içine [0.01061919] 0.0
yahu yorum yaparken bazı şeyleri akıl olumsuz yorum diye film hakları idi de marvel bu hakları alınca bize eski unutun o küçük işler yapan ezik bi spider ama şimdi stark tarafından artık demek istiyor ve bu da her süper kahramanın ilk filmi gibi yeni zamanını anlatıyor eminim si çok çok daha iyi olacak ve çok daha güçlü bir spiderman karşımıza çıkacak [0.9719253] 0.0
